In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder \
    .appName("project")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=project>

In [2]:
spark

In [3]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [5]:
documentAssembler = DocumentAssembler() \
    .setInputCol('context') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [6]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("article_type", StringType(), True),
    StructField("np1", StringType(), True),
    StructField("np2", StringType(), True),
    StructField("context", StringType(), True),
    StructField("source", StringType(), True),
    StructField("category", StringType(), True),
    StructField("location", StringType(), True),
    StructField("time", StringType(), True),])

In [8]:
df = spark.read.csv("s3://anly502project/data/part-r-00000",sep = "\t",header=False,schema=schema)
df_2 = spark.read.csv("s3://anly502project/data/part-r-00001",sep = "\t",header=False,schema=schema)

In [9]:
#### Data Schema
df.printSchema()
df_2.printSchema()

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)



In [10]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [11]:
unioned_df = unionAll([df, df_2])

In [12]:
#### show combined
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|article_type|            np1|np2|             context|       source|            category|            location| time|
+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |intlnews topstor ...|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             |    politics topstor|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |      sports topstor| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...| 

In [13]:
from pyspark.sql.functions import split
split_col = split(unioned_df['category'], ',')
unioned_df = unioned_df.withColumn('category', split_col.getItem(0))

In [14]:
split_col_2 = split(unioned_df['category'], ' ')
unioned_df = unioned_df.withColumn('category', split_col_2.getItem(0))

In [15]:
split_col_3 = split(unioned_df['category'], '_')
unioned_df = unioned_df.withColumn('category', split_col_3.getItem(0))

In [16]:
split_col_4 = split(unioned_df['category'], '-')
unioned_df = unioned_df.withColumn('category', split_col_4.getItem(0))

In [17]:
#### Show data frame after filtering the category
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+---------+--------------------+-----+
|article_type|            np1|np2|             context|       source| category|            location| time|
+------------+---------------+---+--------------------+-------------+---------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             | intlnews|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |  topstor|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             | politics|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |  topstor|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |   sports| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...|             |  topstor| canton, ohio uni...|14363|
|     article| 75-minute mark|  E|arg

In [18]:
df_pip = pipeline.fit(unioned_df).transform(unioned_df)

In [20]:
df_pip.show()

+------------+---------------+---+--------------------+-------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|article_type|            np1|np2|             context|       source|     category|            location| time|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|
+------------+---------------+---+--------------------+-------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |     intlnews|      , kerala india|14299|[[document, 0, 17...|[[token, 0, 3, ar...|[[token, 0, 2, ar...|[[token, 0, 2, ar...|[[token, 0, 2, ar...|    [arg, wall, arg]|
|     article|    Carotenoids|  E|arg1 and caroteno.

In [23]:
from pyspark.sql.functions import explode, col

context_words = df_pip.withColumn("exploded_text", explode(col("finished_clean_lemma")))

In [24]:
context_words.columns

['article_type',
 'np1',
 'np2',
 'context',
 'source',
 'category',
 'location',
 'time',
 'document',
 'token',
 'normalized',
 'lemma',
 'clean_lemma',
 'finished_clean_lemma',
 'exploded_text']

In [36]:
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.4.5
Apache Spark version:  2.4.4


In [37]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)\
    .setPoolingLayer(0) # default 0

bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[OK!]


In [111]:
sc.stop()

In [26]:
SparkSession._instantiatedContext = None